In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mne_connectivity import spectral_connectivity_epochs
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline


In [ ]:
# کد کامل برای تست کانکتیویتی روی 7 سوژه

# import functions from different channels. 
from preprocessing import MotorImageryPreprocessor
from config import DATA_CONFIG, EPOCH_CONFIG, FILTER_CONFIG, COMMON_ROIS 
from new_connectivity import EpochConnectivityProcessor

# لیست‌های نهایی برای ذخیره تمام داده‌ها
all_roi_matrices = []
all_labels = []
all_subjects = []  # برای ردیابی اینکه هر نمونه متعلق به کدام سوژه است

print("شروع پردازش 7 سوژه...")

# حلقه اصلی برای پردازش تمام 7 سوژه (ایندکس 0 تا 6)
for subject_idx in range(7):
    print(f"\n=================== سوژه {subject_idx + 1} (ایندکس {subject_idx}) ===================")
    
    try:
        # 1) پیش‌پردازش برای سوژه فعلی
        prep = (MotorImageryPreprocessor(DATA_CONFIG['data_path'])
                .load_data()
                .set_subject(subject_idx=subject_idx, task_type='imagery')
                .preprocess_signal()
                .bandpass_filter(lowcut=FILTER_CONFIG['lowcut'], 
                               highcut=FILTER_CONFIG['highcut']))
        
        # 2) استخراج epochs برای سوژه فعلی
        hand_epochs, _ = prep.extract_epochs_with_baseline(
            stim_code=EPOCH_CONFIG['stimulus_codes']['hand'],
            epoch_ms=3000
        )
        
        tongue_epochs, _ = prep.extract_epochs_with_baseline(
            stim_code=EPOCH_CONFIG['stimulus_codes']['tongue'],
            epoch_ms=3000
        )
        
        print(f"تعداد epochs دست: {len(hand_epochs)}")
        print(f"تعداد epochs زبان: {len(tongue_epochs)}")
        
        # اطلاعات debugging برای سوژه فعلی
        print("================ DEBUGGING INFO ================")
        srate_valid = prep.srate
        brodmann_labels_valid = prep.brodmann_labels
        print(f"تعداد کل لیبل‌های Brodmann در داده: {len(brodmann_labels_valid)}")
        print(f"تعداد لیبل‌های منحصر به فرد: {len(np.unique(brodmann_labels_valid))}")
        print("==============================================")
        
        # 3) پردازش epochs مربوط به حرکت دست
        print("پردازش epochs دست...")
        for i, epoch_data in enumerate(hand_epochs):
            print(f"  پردازش epoch دست {i+1}/{len(hand_epochs)}")
            
            processor = EpochConnectivityProcessor(
                epoch_data=epoch_data, 
                srate=prep.srate,  # استفاده از srate واقعی
                brodmann_labels=prep.brodmann_labels
            )
            
            final_roi_matrix,feature_names = processor.process(correction_method='subtraction')
            
            all_roi_matrices.append(final_roi_matrix)
            all_labels.append(1)  # لیبل 1 برای دست
            all_subjects.append(subject_idx)
        
        # 4) پردازش epochs مربوط به حرکت زبان
        print("پردازش epochs زبان...")
        for i, epoch_data in enumerate(tongue_epochs):
            print(f"  پردازش epoch زبان {i+1}/{len(tongue_epochs)}")
            
            processor = EpochConnectivityProcessor(
                epoch_data=epoch_data, 
                srate=prep.srate,  # استفاده از srate واقعی
                brodmann_labels=prep.brodmann_labels
            )
            
            final_roi_matrix , feature_names = processor.process(correction_method='subtraction')
            
            all_roi_matrices.append(final_roi_matrix)
            all_labels.append(0)  # لیبل 0 برای زبان
            all_subjects.append(subject_idx)
        
        print(f"سوژه {subject_idx + 1} با موفقیت پردازش شد!")
        
    except Exception as e:
        print(f"خطا در پردازش سوژه {subject_idx + 1}: {str(e)}")
        continue

print("\n=================== خلاصه نهایی ===================")

# تبدیل لیست‌ها به آرایه‌های numpy
X_connectivity_features = np.array(all_roi_matrices)
y = np.array(all_labels)
subjects = np.array(all_subjects)

print(f"شکل نهایی X: {X_connectivity_features.shape}")
print(f"شکل نهایی y: {y.shape}")
print(f"تعداد کل نمونه‌ها: {len(y)}")
print(f"تعداد نمونه‌های دست (لیبل 1): {np.sum(y == 1)}")
print(f"تعداد نمونه‌های زبان (لیبل 0): {np.sum(y == 0)}")

# نمایش توزیع داده‌ها بر اساس سوژه
print("\nتوزیع داده‌ها بر اساس سوژه:")
for subject_idx in range(7):
    subject_mask = subjects == subject_idx
    hand_count = np.sum((subjects == subject_idx) & (y == 1))
    tongue_count = np.sum((subjects == subject_idx) & (y == 0))
    total_count = np.sum(subject_mask)
    print(f"سوژه {subject_idx + 1}: {total_count} نمونه (دست: {hand_count}, زبان: {tongue_count})")
print("🔍 نقشه ویژگی‌ها:")
for i, name in enumerate(feature_names):
    print(f"   ویژگی {i+1}: {name}")

print("\nویژگی‌های معنادار:")
significant_indices = [1, 4, 2, 5, 8, 15]  # از نتایج t-test
for idx in significant_indices:
    print(f"   ویژگی {idx+1}: {feature_names[idx]}")
print("=================== پایان ===================")

# حالا می‌توانید این داده‌ها را برای یادگیری ماشین استفاده کنید
# X_connectivity_features: ویژگی‌های کانکتیویتی
# y: لیبل‌ها (0: زبان، 1: دست)
# subjects: ایندکس سوژه برای هر نمونه

شروع پردازش 7 سوژه...

=================== سوژه 1 (ایندکس 0) ===================
✅ Data loaded successfully. Found 7 subjects.
📊 Subject 0 selected:
   - Sampling rate: 1000 Hz
   - Channels: 46
   - Time points: 376400
   - ROIs found: 12
🔧 DC offset removed from signal.
🔧 Data type converted to float32.
🔧 Bandpass filter applied: 1-120 Hz
📦 Epochs with baseline extracted for stimulus 12:
   - Valid trials: 30/30
   - Baseline: 500 ms (500 samples)
   - Epoch: 3000 ms (3000 samples)
   - Post-trial: 0 ms (0 samples)
   - Total shape: (30, 46, 3500)
📦 Epochs with baseline extracted for stimulus 11:
   - Valid trials: 30/30
   - Baseline: 500 ms (500 samples)
   - Epoch: 3000 ms (3000 samples)
   - Post-trial: 0 ms (0 samples)
   - Total shape: (30, 46, 3500)
تعداد epochs دست: 30
تعداد epochs زبان: 30
================ DEBUGGING INFO ================
تعداد کل لیبل‌های Brodmann در داده: 46
تعداد لیبل‌های منحصر به فرد: 12
پردازش epochs دست...
  پردازش epoch دست 1/30
  پردازش epoch دست 2/30

In [15]:
# تحلیل آماری و طبقه‌بندی کانکتیویتی برای داده‌های 7 سوژه
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns


X = X_connectivity_features
y = np.array(all_labels)  
#significant_features = [1, 2, 4, 5, 8, 15]  # ایندکس‌های p<0.05
#X_selected = X_connectivity_features[:, significant_features]
# استفاده از متغیرهای تولید شده از کد قبلی
#X = X_selected  # شکل: (420, n_features) 
   # شکل: (420,)

print("="*70)
print("                    تحلیل داده‌های کانکتیویتی")
print("="*70)

print(f"🔍 بررسی داده‌ها:")
print(f"   • تعداد کل نمونه‌ها: {X.shape[0]}")
print(f"   • تعداد ویژگی‌های کانکتیویتی: {X.shape[1]}")
print(f"   • نمونه‌های دست (لیبل 1): {np.sum(y == 1)}")
print(f"   • نمونه‌های زبان (لیبل 0): {np.sum(y == 0)}")

### 1. تحلیل آماری تفاوت کانکتیویتی بین دست و زبان
# ===========================================================

print(f"\n📊 تحلیل آماری تفاوت کانکتیویتی:")
print("-" * 50)

# جداسازی داده‌ها بر اساس کلاس
hand_data = X[y == 1]  # داده‌های دست
tongue_data = X[y == 0]  # داده‌های زبان

# محاسبه آمار توصیفی
print(f"📈 آمار توصیفی:")
print(f"   دست - میانگین: {np.mean(hand_data):.4f}, انحراف معیار: {np.std(hand_data):.4f}")
print(f"   زبان - میانگین: {np.mean(tongue_data):.4f}, انحراف معیار: {np.std(tongue_data):.4f}")

# تست t-test برای هر ویژگی کانکتیویتی
significant_features = []
p_values = []

print(f"\n🔬 تست t-test برای ویژگی‌های کانکتیویتی:")
for i in range(X.shape[1]):
    # t-test مستقل برای مقایسه دو گروه
    t_stat, p_value = stats.ttest_ind(hand_data[:, i], tongue_data[:, i])
    p_values.append(p_value)
    
    # تعیین معنی‌داری
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else ""
    
    if p_value < 0.05:
        significant_features.append(i)
    
    print(f"   ویژگی {i+1:2d}: t={t_stat:6.3f}, p={p_value:.4f} {significance}")

print(f"\n✅ تعداد ویژگی‌های معنی‌دار (p<0.05): {len(significant_features)} از {X.shape[1]}")

# محاسبه اندازه اثر (Cohen's d) برای ویژگی‌های معنی‌دار
if significant_features:
    print(f"\n📏 اندازه اثر (Cohen's d) برای ویژگی‌های معنی‌دار:")
    for feat_idx in significant_features[:5]:  # نمایش 5 تای اول
        mean_hand = np.mean(hand_data[:, feat_idx])
        mean_tongue = np.mean(tongue_data[:, feat_idx])
        std_pooled = np.sqrt((np.var(hand_data[:, feat_idx]) + np.var(tongue_data[:, feat_idx])) / 2)
        cohens_d = (mean_hand - mean_tongue) / std_pooled
        print(f"   ویژگی {feat_idx+1}: Cohen's d = {cohens_d:.3f}")


                    تحلیل داده‌های کانکتیویتی
🔍 بررسی داده‌ها:
   • تعداد کل نمونه‌ها: 420
   • تعداد ویژگی‌های کانکتیویتی: 21
   • نمونه‌های دست (لیبل 1): 210
   • نمونه‌های زبان (لیبل 0): 210

📊 تحلیل آماری تفاوت کانکتیویتی:
--------------------------------------------------
📈 آمار توصیفی:
   دست - میانگین: -0.0010, انحراف معیار: 0.0635
   زبان - میانگین: -0.0030, انحراف معیار: 0.0678

🔬 تست t-test برای ویژگی‌های کانکتیویتی:
   ویژگی  1: t= 1.670, p=0.0958 
   ویژگی  2: t= 4.428, p=0.0000 ***
   ویژگی  3: t= 2.063, p=0.0397 *
   ویژگی  4: t= 0.759, p=0.4486 
   ویژگی  5: t= 3.931, p=0.0001 ***
   ویژگی  6: t= 2.505, p=0.0126 *
   ویژگی  7: t=-0.569, p=0.5699 
   ویژگی  8: t= 0.497, p=0.6198 
   ویژگی  9: t=-2.807, p=0.0052 **
   ویژگی 10: t=-0.836, p=0.4034 
   ویژگی 11: t=-0.344, p=0.7306 
   ویژگی 12: t=-0.040, p=0.9683 
   ویژگی 13: t=-1.912, p=0.0565 
   ویژگی 14: t=-1.774, p=0.0768 
   ویژگی 15: t= 0.636, p=0.5248 
   ویژگی 16: t=-2.190, p=0.0291 *
   ویژگی 17: t=-0.338, p=0.735

In [16]:

### 2. طبقه‌بندی با Random Forest
# =================================

print(f"\n" + "="*70)
print("                    طبقه‌بندی با Random Forest")
print("="*70)

print(f"🤖 خلاصه ویژگی‌ها:")
print(f"   • ویژگی‌های کانکتیویتی: {X.shape[1]}")
print(f"   • شکل ماتریس نهایی: {X.shape}")

# محاسبه تنوع ویژگی‌ها
print(f"\n🔍 تنوع ویژگی‌ها:")
for i in range(min(10, X.shape[1])):  # نمایش 10 تای اول
    feature_std = np.std(X[:, i])
    feature_mean = np.mean(X[:, i])
    print(f"   • ویژگی {i+1:2d}: میانگین={feature_mean:.4f}, انحراف معیار={feature_std:.4f}")

if X.shape[1] > 10:
    print(f"   • ... و {X.shape[1] - 10} ویژگی دیگر")

print(f"\n✅ آماده برای طبقه‌بندی!")

### 3. تقسیم داده‌ها بدون Data Leakage
# =====================================

print(f"\n🚨 جلوگیری از Data Leakage - تقسیم صحیح داده‌ها...")

# تقسیم اولیه: 80% تمرین+اعتبارسنجی، 20% تست
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 خلاصه تقسیم داده:")
print(f"   • تمرین+اعتبارسنجی: {len(X_temp)} نمونه")
print(f"   • تست (نگه‌داری شده): {len(X_test)} نمونه")
print(f"   • داده‌های تست هرگز در تمرین استفاده نمی‌شوند!")

### 4. تنظیم مدل Random Forest
# ==============================

# تعریف pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# تعریف گرید پارامترها (ساده‌تر برای سرعت بیشتر)
param_grid = {
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [5, 10, None],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2]
}

# تنظیم Cross-Validation
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

n_combinations = (len(param_grid['rf__n_estimators']) * len(param_grid['rf__max_depth']) * 
                 len(param_grid['rf__min_samples_split']) * len(param_grid['rf__min_samples_leaf']))

print(f"\n🔄 اجرای GridSearchCV فقط روی داده‌های تمرین+اعتبارسنجی...")
print(f"   • جستجو در {n_combinations} ترکیب پارامتر")
print(f"   • استفاده از 5-fold cross-validation")

grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=inner_cv,
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=1
)

# آموزش فقط روی داده‌های تمرین+اعتبارسنجی
grid_search.fit(X_temp, y_temp)

### 5. ارزیابی مدل بدون Data Leakage
# ===================================

print("\n" + "="*70)
print("                    نتایج بدون Data Leakage")
print("="*70)

# بهترین مدل از GridSearch
best_model = grid_search.best_estimator_

print(f"🏆 بهترین امتیاز Cross-Validation: {grid_search.best_score_:.3f}")
print(f"\n🔥 بهترین پارامترها:")
for param, value in grid_search.best_params_.items():
    param_name = param.replace('rf__', '')
    print(f"   • {param_name}: {value}")

# ارزیابی نهایی روی داده‌های تست
print(f"\n🧪 ارزیابی نهایی روی داده‌های تست:")
test_predictions = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)

print(f"   • دقت تست: {test_accuracy:.3f}")
print(f"   • این تخمین واقعی عملکرد است!")

# تحلیل اهمیت ویژگی‌ها
rf_model = best_model.named_steps['rf']
feature_importance = rf_model.feature_importances_

print(f"\n🔍 تحلیل اهمیت ویژگی‌ها:")
# نمایش 10 ویژگی مهم
top_indices = np.argsort(feature_importance)[-10:][::-1]
for i, feat_idx in enumerate(top_indices):
    print(f"   {i+1:2d}. ویژگی {feat_idx+1}: {feature_importance[feat_idx]:.4f}")

# نتایج تفصیلی روی داده‌های تست
print(f"\n📋 عملکرد تفصیلی روی داده‌های تست:")
target_names = ['زبان', 'دست']
print("\nگزارش طبقه‌بندی:")
print(classification_report(y_test, test_predictions, target_names=target_names, zero_division=0))

print("\nماتریس درهم‌ریختگی:")
cm = confusion_matrix(y_test, test_predictions)
print(f"              پیش‌بینی")
print(f"           زبان  دست")
print(f"واقعی زبان  {cm[0,0]:3d}   {cm[0,1]:3d}")
print(f"      دست  {cm[1,0]:3d}   {cm[1,1]:3d}")

# نمونه پیش‌بینی‌ها
print(f"\n🔬 نمونه پیش‌بینی‌های تست:")
n_samples = min(10, len(X_test))
for i in range(n_samples):
    pred_label = "دست" if test_predictions[i] == 1 else "زبان"
    actual_label = "دست" if y_test[i] == 1 else "زبان"
    status = "✅" if test_predictions[i] == y_test[i] else "❌"
    print(f"   {status} نمونه {i+1:2d}: پیش‌بینی={pred_label:4s}, واقعی={actual_label:4s}")

### 6. خلاصه نهایی
# ================

print("\n" + "="*70)
print("                         خلاصه نهایی")
print("="*70)
print(f"🎯 وظیفه: طبقه‌بندی تصور حرکت دست در مقابل زبان")
print(f"📊 ویژگی‌ها: {X.shape[1]} ویژگی کانکتیویتی")
print(f"📈 نمونه‌ها: {X.shape[0]} نمونه ({np.sum(y==1)} دست، {np.sum(y==0)} زبان)")
print(f"")
print(f"🔬 عملکرد مدل:")
print(f"   • دقت Cross-Validation: {grid_search.best_score_:.3f}")
print(f"   • دقت داده‌های تست: {test_accuracy:.3f}")
print(f"")
print(f"✅ Data Leakage: جلوگیری شد")
print(f"✅ مدل: Random Forest با اعتبارسنجی صحیح")
print(f"📋 تخمین قابل اعتماد: دقت تست = {test_accuracy:.3f}")
print("="*70)


                    طبقه‌بندی با Random Forest
🤖 خلاصه ویژگی‌ها:
   • ویژگی‌های کانکتیویتی: 21
   • شکل ماتریس نهایی: (420, 21)

🔍 تنوع ویژگی‌ها:
   • ویژگی  1: میانگین=-0.0029, انحراف معیار=0.0475
   • ویژگی  2: میانگین=-0.0031, انحراف معیار=0.0388
   • ویژگی  3: میانگین=-0.0046, انحراف معیار=0.0653
   • ویژگی  4: میانگین=-0.0027, انحراف معیار=0.0863
   • ویژگی  5: میانگین=0.0001, انحراف معیار=0.0800
   • ویژگی  6: میانگین=-0.0040, انحراف معیار=0.0557
   • ویژگی  7: میانگین=-0.0066, انحراف معیار=0.0312
   • ویژگی  8: میانگین=-0.0072, انحراف معیار=0.0470
   • ویژگی  9: میانگین=-0.0077, انحراف معیار=0.0525
   • ویژگی 10: میانگین=-0.0058, انحراف معیار=0.0464
   • ... و 11 ویژگی دیگر

✅ آماده برای طبقه‌بندی!

🚨 جلوگیری از Data Leakage - تقسیم صحیح داده‌ها...
📊 خلاصه تقسیم داده:
   • تمرین+اعتبارسنجی: 336 نمونه
   • تست (نگه‌داری شده): 84 نمونه
   • داده‌های تست هرگز در تمرین استفاده نمی‌شوند!

🔄 اجرای GridSearchCV فقط روی داده‌های تمرین+اعتبارسنجی...
   • جستجو در 36 ترکیب پارامتر
   • ا